In [1]:
from IPython.display import IFrame

IFrame('Boyce_12e_PPT_ch05_3.pdf', width=1000, height=700)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown
from scipy.integrate import solve_ivp
from matplotlib.patches import Rectangle
sns.set_context('poster')
import warnings
warnings.filterwarnings("ignore")

class SeriesSolutionPlayDemo:
    def __init__(self):
        self.t_min, self.t_max, self.n_points = -5.0, 5.0, 1000
        self.x_vals = np.linspace(self.t_min, self.t_max, self.n_points)
        self.n_terms = 8
        self.y0 = 1.0
        self.dy0 = 0.0
        self.x0 = 0.0
        self._compute_radius()
        self.solve()
        self._create_widgets()

    def _compute_radius(self):
        self.radius = min(abs(self.x0 - 1), abs(self.x0 + 1))

    def solve(self):
        x0 = self.x0
        n_terms = self.n_terms
        a = [self.y0, self.dy0]
        for n in range(2, n_terms+1):
            denom = (n)*(n-1)
            if abs(1 - x0**2) < 1e-13:
                a_next = 0.0
            else:
                a_next = -a[n-2] / (denom * (1 - x0**2))
            a.append(a_next)
        self.coeffs = np.array(a)
        dx = self.x_vals - x0
        powers = np.vstack([dx**k for k in range(len(self.coeffs))])
        self.series_vals = np.dot(self.coeffs, powers)

        radius = self.radius
        mask_in_radius = np.abs(self.x_vals - x0) < (radius - 1e-6)
        x_for_ivp = self.x_vals[mask_in_radius]
        if len(x_for_ivp) < 2:
            self.analytic_vals = np.nan * np.ones_like(self.x_vals)
            return
        def ode(t, y):
            fac = 1/(1-t**2) if abs(1-t**2) >= 1e-13 else 0
            return [y[1], -fac * y[0]]
        try:
            analytic_sol = solve_ivp(
                ode, [x_for_ivp[0], x_for_ivp[-1]], [self.y0, self.dy0],
                t_eval=x_for_ivp, rtol=2e-8, atol=1e-10
            )
            analytic_vals = np.nan * np.ones_like(self.x_vals)
            if analytic_sol.status == 0 or analytic_sol.status == 1:
                analytic_vals[mask_in_radius] = analytic_sol.y[0]
            self.analytic_vals = analytic_vals
        except Exception:
            self.analytic_vals = np.nan * np.ones_like(self.x_vals)

    def _create_widgets(self):
        self.n_slider = widgets.IntSlider(
            min=2, max=30, value=8, step=2,
            description="Series Terms (n):",
            layout=widgets.Layout(width='330px'),
            style={'description_width': 'initial'},
            continuous_update=False
        )
        self.x0_slider = widgets.FloatSlider(
            min=-0.99, max=0.99, value=0.0, step=0.01,
            description="Ordinary point x0:",
            layout=widgets.Layout(width='350px'),
            style={'description_width': 'initial'},
            continuous_update=False
        )
        self.play_widget = widgets.Play(
            value=0, min=0, max=self.n_points-1, step=1, interval=30
        )
        self.play_widget.layout.display = 'none'
        self.play_btn  = widgets.Button(description='▶ Play',  button_style='success', layout=widgets.Layout(width='85px'))
        self.pause_btn = widgets.Button(description='⏸ Pause', button_style='warning',layout=widgets.Layout(width='85px'))
        self.stop_btn  = widgets.Button(description='⏹ Stop',  button_style='danger', layout=widgets.Layout(width='85px'))
        self.reset_btn = widgets.Button(description='⟲ Reset', button_style='info', layout=widgets.Layout(width='85px'))
        self.play_btn.on_click(lambda *_: setattr(self.play_widget, 'playing', True))
        self.pause_btn.on_click(lambda *_: setattr(self.play_widget, 'playing', False))
        self.stop_btn.on_click(self._stop)
        self.reset_btn.on_click(lambda *_: setattr(self.play_widget, 'value', 0))
        self.x_slider = widgets.FloatSlider(
            value=self.x_vals[0], min=self.t_min, max=self.t_max,
            step=(self.t_max - self.t_min)/(self.n_points-1),
            description='x:', disabled=True,
            layout=widgets.Layout(width="600px"),
            style={'description_width':'initial'}
        )
        def _param_change(*_):
            self.n_terms = self.n_slider.value
            self.x0 = self.x0_slider.value
            self._compute_radius()
            self.solve()
            self._stop()
        self.n_slider.observe(_param_change, names='value')
        self.x0_slider.observe(_param_change, names='value')

    def _stop(self, *_):
        self.play_widget.playing = False
        self.play_widget.value = 0

    def _update(self, frame_idx):
        i = int(frame_idx)
        x = self.x_vals
        y_series = self.series_vals
        y_true = self.analytic_vals
        error = np.abs(y_series - y_true)
        x0 = self.x0
        radius = self.radius
        xi = x[i]
        self.x_slider.value = xi
        fig, axes = plt.subplots(1,2, figsize=(17,6), gridspec_kw={'width_ratios': [1.2, 1]}, sharex=True)
        ax = axes[0]
        ax.plot(x, y_true, '--', lw=2, color='green', label='Numerical Solution')
        ax.plot(x, y_series, color='#196BBD', lw=2, label=f"Series (n={self.n_terms})")
        ax.plot(x[:i+1], y_series[:i+1], color='crimson', lw=4, alpha=.85, label='Traced Series')
        if np.isfinite(radius):
            ax.add_patch(Rectangle((x0-radius, -8), 2*radius, 16, color='royalblue', alpha=0.09, zorder=0))
            ax.axvline(x0-radius, color="royalblue", ls=':', lw=2, alpha=0.75)
            ax.axvline(x0+radius, color="royalblue", ls=':', lw=2, alpha=0.75)
            ax.text(x0, 7.1, f"Radius of Convergence = {radius:.2f}", color="royalblue", fontsize=14, ha='center', va='bottom')
        if np.isfinite(radius) and abs(xi - x0) > radius:
            ax.plot(xi, y_series[i], marker='o', color='magenta', ms=17, markeredgecolor='k', zorder=10)
        else:
            ax.plot(xi, y_series[i], marker='o', color='crimson', ms=10, zorder=10)
        ax.set_xlim(self.t_min, self.t_max)
        ax.set_ylim(-8, 8)
        ax.set_xlabel('x', fontsize=18)
        ax.set_ylabel('y(x)', fontsize=18)
        ax.axhline(0, color='k', lw=0.75)
        ax.axvline(0, color='grey', lw=0.75)
        ax.grid(True, alpha=0.22)
        ax.legend(loc='upper right', fontsize=13)
        ax2 = axes[1]
        ax2.plot(x, error, color='orange', lw=3, label="|Series - Numerical|")
        ax2.fill_between(x, 0, error, color='orange', alpha=0.22)
        ax2.set_xlim(self.t_min, self.t_max)
        ax2.set_ylim(0, min(max(0.02, np.nanmax(error)*1.13), 8))
        ax2.set_xlabel('x', fontsize=16)
        ax2.set_title("Error vs. x", fontsize=16)
        ax2.grid(True, alpha=0.2)
        ax2.legend(fontsize=12, loc='upper left')
        if np.isfinite(radius):
            for a in [ax, ax2]:
                a.axvline(x0-radius, color="royalblue", ls=':', lw=2, alpha=0.75)
                a.axvline(x0+radius, color="royalblue", ls=':', lw=2, alpha=0.75)
        if np.isfinite(radius) and abs(xi - x0) > radius:
            fig.suptitle("Warning: x is beyond the radius of convergence!", color='magenta', fontsize=22, fontweight='bold', y=1.04)
        plt.tight_layout()
        plt.show()

    def display(self):
        descr = (
            "<b>Equation:</b> y'' + 1/(1-x^2) y = 0<br>"
            "This demo shows the Taylor series approximation for this ODE, centered at x0.<ul>"
            "<li>Select number of terms (n) and the ordinary point x0 (the series center).</li>"
            "<li>The blue region is the radius of convergence (distance to nearest singularity at x=1 or x=-1).</li>"
            "<li>If the traced point leaves this region, it turns bold magenta and a warning will appear.</li>"
            "<li>Initial conditions: y(x0)=1, y'(x0)=0.</li>"
            "</ul>"
        )
        display(Markdown(descr))
        param_box = widgets.HBox([self.n_slider, self.x0_slider])
        controls = widgets.HBox([
            self.play_btn, self.pause_btn, self.stop_btn, self.reset_btn
        ])
        display(param_box, controls, self.x_slider)
        display(self.play_widget)
        out = widgets.interactive_output(self._update, {'frame_idx': self.play_widget})
        display(out)
        self._stop()

def run_demo():
    demo = SeriesSolutionPlayDemo()
    demo.display()

run_demo()

<b>Equation:</b> y'' + 1/(1-x^2) y = 0<br>This demo shows the Taylor series approximation for this ODE, centered at x0.<ul><li>Select number of terms (n) and the ordinary point x0 (the series center).</li><li>The blue region is the radius of convergence (distance to nearest singularity at x=1 or x=-1).</li><li>If the traced point leaves this region, it turns bold magenta and a warning will appear.</li><li>Initial conditions: y(x0)=1, y'(x0)=0.</li></ul>

FloatSlider(value=-2.0, description='x:', disabled=True, layout=Layout(width='600px'), max=2.0, min=-2.0, step…

Play(value=0, interval=30, layout=Layout(display='none'), max=999)

Output()